### Install the Indexify Extractor SDK, Langchain Retriever and the Indexify Client

In [1]:
%%capture
!pip install indexify-extractor-sdk indexify-langchain indexify

### Start the Indexify Server

In [ ]:
!./indexify server -d

### Download an Embedding Extractor
On another terminal we'll download and start the embedding extractor which we will use to index text from the Rental pdf document.

In [ ]:
!indexify-extractor download hub://embedding/minilm-l6
!indexify-extractor join-server

### Download an Chunking Extractor
On another terminal we'll download and start the chunking extractor that will create chunks from the text and embeddings.

In [ ]:
!indexify-extractor download hub://text/chunking
!indexify-extractor join-server

### Download the Markdown Extractor
On another terminal we'll install the necessary dependencies and start the Markdown extractor which we will use to get text out of Rental PDF documents in a markdown format.

Download and start the Markdown extractor

In [ ]:
!indexify-extractor download hub://pdf/markdown
!indexify-extractor join-server

### Download the Rental PDF

In [3]:
import requests
req = requests.get("https://www.timescar-rental.com/pdf/agreement/en_agreement_200401.pdf")

with open('en_agreement_200401.pdf','wb') as f:
    f.write(req.content)

### Test the extractors

We will try MarkdownExtractor first. The MarkdownExtractor can extract all the values from text in one shot and passes it to the next chained extractors as a markdown formatted document which can be used for question answering.

In [4]:
from indexify_extractor_sdk import load_extractor, Content

mdextractor, mdconfig_cls = load_extractor("markdown.markdown_extractor:MarkdownExtractor")
content = Content.from_file("en_agreement_200401.pdf")

md_result = mdextractor.extract(content)
text_content = next(content.data.decode('utf-8') for content in md_result)
text_content

/Users/rishiraj/miniconda3/envs/tensorlake/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded texify model to mps with torch.float16 dtype


/Users/rishiraj/miniconda3/envs/tensorlake/lib/python3.10/site-packages/transformers/modeling_utils.py:962: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


'\n## Times Car Rental Terms And Conditions\n\nThis Agreement shall take effect starting April 1st, 2020. \n\n## Chapter 1 ‒ General Provisions Article 1: Application Of Agreement\n\n1. \nTIMES MOBILITY CO., LTD. (hereinafter referred to as the "Company") shall rent out a vehicle (hereinafter referred to \nas "Rental Car") to the Renter, and the Renter shall rent the Rental Car from the Company, all in accordance with the \nprovisions in this Agreement. Matters not provided for in this Agreement shall be governed by relevant laws and regulations, and common practice. \n2. \nThe Company may make special provisions as long as they do not violate the purport of this Agreement, laws and regulations, administrative issuances, and common practice. If a special provision is made, such provision shall take precedence over this Agreement. \n3. \nIf the Renter has designated another Driver as described in Article 7, Paragraph 1, then the Renter shall ensure that the other Driver understands and 

### Create Extraction Policies
Instantiate the Indexify Client

In [ ]:
from indexify import IndexifyClient
client = IndexifyClient()

First, create a policy to get texts and contents out of the Rental PDF.

In [ ]:
client.add_extraction_policy(extractor='tensorlake/markdown-extractor', name="md-extraction")

Second, from the texts and contents create an embedding based index.

In [ ]:
client.add_extraction_policy(extractor='tensorlake/chunk-extractor', name="chunks", content_source="pdf-extraction", input_params={"chunk_size": 512, "overlap": 150})

In [ ]:
client.add_extraction_policy(extractor='tensorlake/minilm-l6', name="get-embeddings", content_source="chunks")

### Upload a Rental PDF File

In [ ]:
client.upload_file(path="en_agreement_200401.pdf")

### What is happening behind the scenes

Indexify is designed to seamlessly respond to ingestion events by assessing all existing policies and triggering the necessary extractors for extraction. Once the PDF extractor completes the process of extracting texts, bytes, and JSONs from the document, it automatically initiates the embedding extractor to chunk the content, extract embeddings, and populate an index.

With Indexify, you have the ability to upload hundreds of Rental PDF files simultaneously, and the platform will efficiently handle the extraction and indexing of the contents without requiring manual intervention. To expedite the extraction process, you can deploy multiple instances of the extractors, and Indexify's built-in scheduler will transparently distribute the workload among them, ensuring optimal performance and efficiency.

### Perform RAG
Initialize the Langchain Retreiver.

In [ ]:
from indexify_langchain import IndexifyRetriever
params = {"name": "get-embeddings.embedding", "top_k": 3}
retriever = IndexifyRetriever(client=client, params=params)

Now create a chain to prompt OpenAI with data retreived from Indexify to create a simple Q&A bot

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

Now ask any question related to the ingested Rental PDF document

In [ ]:
chain.invoke("Who will be responsible for damages not compensated by the insurance?")
# The Renter and the Driver shall be responsible for damages not compensated for by the insurance benefit or compensation.